In [32]:
from utils import *
from keras.layers import Input, BatchNormalization, GaussianDropout, concatenate, Dense, PReLU, Dropout, ReLU
from keras.optimizers import Adam, SGD
from keras.initializers import he_normal, glorot_normal
from keras.models import Model

In [4]:
df =aggregate(lb_encode=False)
df = clean_data(df)

--------------------
train & test ( Sat Sep 15 22:31:56 2018 )
--------------------
['department', 'region', 'education', 'gender', 'recruitment_channel', 'no_of_trainings', 'previous_year_rating', 'KPIs_met >80%', 'awards_won?', 'length_of_service', 'age', 'KPI_award', 'region_channel', 'education_region']
     DF shape: (78292, 443)
After removing empty features there are 439 features
After removing features with the same distribution on 0 and 1 classes there are 216 features
After removing features with not the same distribution on train and test datasets there are 216 features
After removing features which are having information value less than 0.02 and greater than 0.8s 211 features


In [5]:
df.columns = ['_'.join(c.split()) for c in df.columns]
df.columns = [c.replace("'", '_') for c in df.columns]
df.columns = [c.replace("&", '_') for c in df.columns]
df.columns = [c.replace(">", '_') for c in df.columns]
df.columns = [c.replace("%", '_') for c in df.columns]
df.columns = [c.replace("?", '_') for c in df.columns]

In [55]:
feats = [c for c in df.columns if c not in ['is_promoted', 'employee_id']]
corrupt_df = df_inputSwapNoise(df[feats], p = 0.2)
corrupt_df = corrupt_df.apply(rank_gauss, axis = 1)
df_feats=df[feats].apply(rank_gauss, axis = 1)

In [56]:
def prep_keras_data(df):
    X = []
    for col in df.columns:
        if col not in ['employee_id', 'is_promoted']:
            X.append(np.array(df[col]))
    return X

In [66]:
def DAE(inp_layers_name):
    inp_layers = []
    for i in inp_layers_name:
        inp_layers.append(Input(shape = [1]))
    
    x = concatenate(inp_layers)
    x = Dense(units=1500, kernel_initializer='glorot_normal')(x)
    x = ReLU()(x)
    x = Dropout(0.5)(x)
    
    enc1 = Dense(units=1500, kernel_initializer='glorot_normal')(x)
    x = ReLU()(enc1)
    x = Dropout(0.5)(x)
    
    enc2 = Dense(units=1500, kernel_initializer='glorot_normal')(x)
    x = ReLU()(enc2)
    x = Dropout(0.5)(x)

    dec = Dense(len(inp_layers_name))(x)
    
    model = Model(inp_layers, dec)
    enc = Model(inp_layers, concatenate([enc1, enc2]))
    optim = SGD(lr=0.0003, decay=0.995, momentum=0.99, clipnorm = 1)
    #optim = Adam(lr=0.0003, decay=0.995, clipvalue = 0.1)
    model.compile(optimizer = optim, loss='mean_squared_error')
    
    return model, enc

In [67]:
input_data = prep_keras_data(corrupt_df)

inp_names = [c for c in corrupt_df.columns if c not in ['employee_id', 'is_promoted']]

output = df_feats[inp_names].values

In [71]:
BATCH_SIZE = 128
epochs = 20
model, enc = DAE(inp_names)

model.fit(input_data, output, batch_size=BATCH_SIZE, epochs= 200)

Epoch 1/200
78292/78292 [==============================] - 67s 861us/step - loss: 0.5799
Epoch 2/200
78292/78292 [==============================] - 64s 821us/step - loss: 0.5270
Epoch 3/200
78292/78292 [==============================] - 65s 826us/step - loss: 0.5166
Epoch 4/200
78292/78292 [==============================] - 63s 805us/step - loss: 0.51061
Epoch 5/200
78292/78292 [==============================] - 71s 901us/step - loss: 0.5061
Epoch 6/200
78292/78292 [==============================] - 64s 815us/step - loss: 0.5027
Epoch 7/200
78292/78292 [==============================] - 62s 797us/step - loss: 0.4999
Epoch 8/200
78292/78292 [==============================] - 66s 849us/step - loss: 0.4977
Epoch 9/200
78292/78292 [==============================] - 64s 819us/step - loss: 0.4958
Epoch 10/200
78292/78292 [==============================] - 63s 809us/step - loss: 0.4938
Epoch 11/200
78292/78292 [==============================] - 70s 898us/step - loss: 0.4925
Epoch 12/200
78292

78292/78292 [==============================] - 53s 675us/step - loss: 0.4675
Epoch 93/200
78292/78292 [==============================] - 53s 676us/step - loss: 0.4674
Epoch 94/200
78292/78292 [==============================] - 53s 671us/step - loss: 0.4675
Epoch 95/200
78292/78292 [==============================] - 53s 675us/step - loss: 0.4674
Epoch 96/200
78292/78292 [==============================] - 53s 675us/step - loss: 0.4675
Epoch 97/200
78292/78292 [==============================] - 53s 677us/step - loss: 0.4674
Epoch 98/200
78292/78292 [==============================] - 53s 672us/step - loss: 0.4671
Epoch 99/200
78292/78292 [==============================] - 53s 678us/step - loss: 0.4671
Epoch 100/200
78292/78292 [==============================] - 53s 671us/step - loss: 0.4671
Epoch 101/200
78292/78292 [==============================] - 53s 675us/step - loss: 0.4670
Epoch 102/200
78292/78292 [==============================] - 53s 673us/step - loss: 0.4669
Epoch 103/200
78292/

78292/78292 [==============================] - 137s 2ms/step - loss: 0.4644
Epoch 183/200
78292/78292 [==============================] - 145s 2ms/step - loss: 0.4645
Epoch 184/200
78292/78292 [==============================] - 139s 2ms/step - loss: 0.4644
Epoch 185/200
78292/78292 [==============================] - 136s 2ms/step - loss: 0.4643ETA: 2s - l
Epoch 186/200
78292/78292 [==============================] - 132s 2ms/step - loss: 0.4641
Epoch 187/200
78292/78292 [==============================] - 83s 1ms/step - loss: 0.4643
Epoch 188/200
78292/78292 [==============================] - 78s 994us/step - loss: 0.4643
Epoch 189/200
78292/78292 [==============================] - 69s 876us/step - loss: 0.4643
Epoch 190/200
78292/78292 [==============================] - 94s 1ms/step - loss: 0.4643
Epoch 191/200
78292/78292 [==============================] - 68s 864us/step - loss: 0.4642
Epoch 192/200
78292/78292 [==============================] - 78s 991us/step - loss: 0.4641
Epoch 193/2

In [73]:
emb = enc.predict(input_data)
emb.shape

(78292, 3000)

In [78]:
def denseNN():
    inp_layers = []
    for i in range(3000):
        inp_layers.append(Input([1]))
    
    x = concatenate(inp_layers)
    x = Dense(4500, kernel_initializer='glorot_normal')(x)
    x = ReLU()(x)
    x = Dropout(0.4)(x)
    
    x = Dense(4500, kernel_initializer='glorot_normal')(x)
    x = ReLU()(x)
    x = Dropout(0.4)(x)
    
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp_layers, out)
    optim = SGD(lr=0.0003, decay=0.995, momentum=0.99, clipnorm = 1)
    model.compile(loss = 'binary_crossentropy', optimizer = optim)
    
    return model

In [79]:
model = denseNN()

In [80]:
df_train = pd.read_csv('../input/train_LZdllcl.csv')
df_train.shape

(54808, 14)

In [89]:
inp_data = emb[:54808, :]
test_data = emb[54808:, :]
inp_data_list = []
test_data_list = []
for i in range(3000):
    inp_data_list.append(inp_data[:, i])
    test_data_list.append(test_data[:, i])
    
y = np.array(df_train['is_promoted'])

In [91]:
from sklearn.model_selection import KFold
cvlist = KFold(n_splits=5, random_state=11).split(df_train)

In [ ]:
for run in range(3):
    for tr_idx, val_idx in cvlist:
        oof_preds = np.zeros(inp_data.shape[0])
        preds = []
        tr_data, val_data = inp_data[tr_idx, :], inp_data[val_idx, :]
        y_tr, y_val = y[tr_idx], y[val_idx]
        tr_data_list = []
        val_data_list = []
        for i in range(3000):
            tr_data_list.append(tr_data[:, i])
            val_data_list.append(val_data[:, i])

        model.fit(tr_data_list, y_tr, batch_size=128, epochs= 50, validation_data= (val_data_list, y_val))
        oof_preds[val_idx] = model.predict(val_data_list)
        preds.append(model.predict(test_data_list))
        np.mean(preds, axis = 0)
    np.save(f'../oof_preds_{run}.npy', oof_preds)
    np.save(f'../test_preds_{run}.npy', preds)

Train on 43846 samples, validate on 10962 samples
Epoch 1/50
43846/43846 [==============================] - 223s 5ms/step - loss: 0.3352 - val_loss: 0.3041
Epoch 2/50
43846/43846 [==============================] - 243s 6ms/step - loss: 0.2942 - val_loss: 0.3017
Epoch 3/50
43846/43846 [==============================] - 228s 5ms/step - loss: 0.2921 - val_loss: 0.3014
Epoch 4/50
43846/43846 [==============================] - 224s 5ms/step - loss: 0.2915 - val_loss: 0.3013
Epoch 5/50
43846/43846 [==============================] - 223s 5ms/step - loss: 0.2910 - val_loss: 0.3012
Epoch 6/50
22656/43846 [==============>...............] - ETA: 1:46 - loss: 0.2915